In [86]:
import math
import torch
from torch.autograd.gradcheck import zero_gradients
from torch.autograd import grad
from projects.NeuralForceField.train import *
from projects.NeuralForceField.graphs import * 
import numpy as np

def compute_jacobian(inputs, output, device):
    """
    inputs: some variable inputs of dimesnion ( , N) that requires grad
    output: has the same dimension as inputs
    return: jacobian: dimension 
    """
    assert inputs.requires_grad

    num_classes = output.size()[1] # if the input data is batched num_classes = output.size()[1] 

    jacobian = torch.zeros(num_classes, *inputs.size())
    grad_output = torch.zeros(*output.size())
    if inputs.is_cuda:
        grad_output = grad_output.to(device)
        jacobian = jacobian.to(device)

    for i in range(num_classes):
        zero_gradients(inputs)
        grad_output.zero_()
        grad_output[:, i] = 1
        output.backward(grad_output, retain_graph=True)
        jacobian[i] = inputs.grad.data

    return torch.transpose(jacobian, dim0=0, dim1=1)

def compute_grad(inputs, output):
    '''
    input:  some variable inputs of dimesnion ( , N) that requires grad
    output: some scalar output
    return: gradients of every input with respect to output, has the same dimension as the input
    '''
    assert inputs.requires_grad
    
    gradspred, = grad(output, inputs, grad_outputs=output.data.new(output.shape).fill_(1),
                   create_graph=True, retain_graph=True)
    
    return gradspred

def compute_hess(inputs, output, device):
    '''
    input:  some variable inputs of dimesnion ( , N) that requires grad
    output: some scalar output
    return: Hessian of the output with respect to inputs, it is has diemension N by N
    '''
    gradient = compute_grad(inputs, output)
    print(inputs.shape)
    print(gradient.shape)
    hess = compute_jacobian(inputs, gradient, device=device)
    
    return hess

In [98]:
# ethanol data 
train = np.load('/home/wwj/data/ethanol_ccsd_t-train.npz')
xyz_data = np.dstack((np.array([train.f.z]*1000).reshape(1000, -1, 1), np.array(train.f.R)))
force_data = train.f.F
energy_data = train.f.E.squeeze()

graph_data = load_graph_data(xyz_data=xyz_data, energy_data=energy_data, batch_size=2, cutoff=5.0,
                             force_data=force_data, au_flag=False, subtract_mean_flag = True)

In [99]:
# initialize parameters
par = dict()

par["n_filters"] = 256
par["n_gaussians"] = 32
par["n_atom_basis"] = 256
par["optim"] = 1e-4
par["scheduler"] = True
par["train_percentage"] = 0.8
par["T"] = 10
par["batch_size"] = 2
par["cutoff"] = 5.0 
par["max_epoch"] = 1000
par["trainable_gauss"] = True
par["rho"] = 0.1
par["eps"] = 1e-5

In [100]:
model = Model(par=par, graph_data=graph_data, device=3, job_name="test", graph_batching=False, root='./log/')

In [114]:
xyz, a, r, f, u, N = model.parse_batch(1)
#xyz.requires_grad = True

xyz_reshape = xyz.reshape(2,27)
xyz_reshape.requires_grad = True
xyz_input = xyz_reshape.reshape(2,9,3)
r = r.reshape(2,9)

U = model.model(r=r, xyz=xyz_input)
#f_pred = -compute_grad(inputs=xyz_reshape, output=U)

hess = compute_hess(inputs=xyz_reshape, output=U, device=3)

torch.Size([2, 27])
torch.Size([2, 27])


torch.Size([2, 27, 27])

In [49]:
hess.shape

torch.Size([9, 3, 3])

In [78]:
xyz_input.shape

torch.Size([9, 3])

In [79]:
U.shape

torch.Size([1, 1])

In [95]:
r.shape

torch.Size([9, 1])